# <font color='red'>Maximize Flight Profit</font>

1. Here we are using linear programming to maximizing the profit
2. First we are iportiong the required libraries
3. Read all the text files required for the calculation
4. For data [flight data](http://mgmt.iisc.ac.in/~parthar/)
5. ppt is given to know more about the problem

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

demand = pd.read_csv('demand.txt', delimiter="\t", header=None,names=["itinerary_name", "demand"])
capacity = pd.read_csv('capacity.txt', delimiter="\t", header=None,names=["leg_name", "capacity"])
fare = pd.read_csv('fare.txt', delimiter="\t", header=None,names=["itinerary_name", "fare"])
it_leg = pd.read_csv('it_leg.txt', delimiter="\t", header=None,names=["itinerary_name", "leg_name"])


### <font color='red'>CREATING DICTIONARIES</font>

1. Itineraries and fligt legs are numberd for easy calculation

In [2]:
itinerary_dict = {}
l=0
for ind in demand.index:
    itinerary_dict[demand['itinerary_name'][ind]] = l
    l=l+1

In [3]:
leg_dict = {}
k=0
for ind in capacity.index:
    leg_dict[capacity['leg_name'][ind]] = k
    k=k+1

### <font color='red'>CREATING MATRIX and ARRAY</font>

In [4]:
Num_inerary,a = demand.shape

In [5]:
Num_flight_legs,s = capacity.shape

In [6]:
A = np.zeros( (Num_inerary+Num_flight_legs, Num_inerary) )

In [7]:
for i in range (Num_inerary):
    A[i,i] = 1
    

In [8]:
for ind in it_leg.index:
    i=itinerary_dict[it_leg['itinerary_name'][ind]]
    j=Num_inerary+leg_dict[it_leg['leg_name'][ind]]
    A[j,i] = 1

In [9]:
d = demand.iloc[:,1].values
c = capacity.iloc[:,1].values
f = fare.iloc[:,1].values 
x = np.zeros((Num_inerary,1))
b = np.zeros((Num_inerary+Num_flight_legs))
fa = np.zeros((Num_inerary))

In [10]:

for i in range(Num_inerary):
    b[i] = d[i]

In [11]:
for i in range(Num_inerary,Num_inerary+Num_flight_legs):
    b[i] = c[i-Num_inerary]

In [12]:
for i in range (Num_inerary):
    fa[i] = -f[i]

### <font color='red'>LINPROG</font>
1. Here we are minimizing the negative of the required function which need to be maximized . 
2. Negative of the minimum gives maximun


In [13]:
res = linprog(c=fa, A_ub=A, b_ub=b,bounds=(0, None))

In [14]:
print (res)
net_profit = -res.fun
print (-res.fun)

     con: array([], dtype=float64)
     fun: -495873.5324180095
 message: 'Optimization terminated successfully.'
     nit: 20
   slack: array([-7.06847081e-12, -4.58022509e-12,  1.27606814e-11, ...,
        7.08493092e+01,  2.97276636e+01,  2.87741991e+02])
  status: 0
 success: True
       x: array([9.22000000e-02, 2.84400000e-01, 1.17510000e+00, ...,
       6.29388957e-04, 8.48273228e-04, 2.43278920e-01])
495873.5324180095


### Duplicate of every table , matrix and array is created for doing the second part of the Assingment

In [15]:
demand_dup = demand
capacity_dup = capacity
fare_dup = fare
it_leg_dup = it_leg
d_dup = demand.iloc[:,1].values
c_dup = capacity.iloc[:,1].values
f_dup = fare.iloc[:,1].values 
x_dup = np.zeros((Num_inerary,1))
b_dup = np.zeros((Num_inerary+Num_flight_legs))
fa_dup = np.zeros((Num_inerary))

### New demand and fares are calculated

In [25]:
sub = 'RHC'
start = 16
for ind in it_leg.index:
    it_leg_dup["Indexes"] = it_leg_dup["leg_name"].str.find(sub, start)


In [26]:
for ind in it_leg_dup.index:
    if (it_leg_dup['Indexes'][ind]==19):
        i = itinerary_dict[it_leg_dup['itinerary_name'][ind]]
        d_dup[i] = d[i]*115/100
        f_dup[i] = f[i]*125/100
        


### <font color='red'>LINPROG</font>

In [18]:
for i in range(Num_inerary):
    b_dup[i] = d_dup[i]
for i in range(Num_inerary,Num_inerary+Num_flight_legs):
    b_dup[i] = c_dup[i-Num_inerary]
for i in range (Num_inerary):
    fa_dup[i] = -f_dup[i]
res_new = linprog(c=fa_dup, A_ub=A, b_ub=b,bounds=(0, None))

In [19]:
print (res_new)
net_profit_new = -res_new.fun
print (-res_new.fun)

     con: array([], dtype=float64)
     fun: -529239.4682509315
 message: 'Optimization terminated successfully.'
     nit: 20
   slack: array([-6.54837295e-13, -3.33288952e-13,  6.79456491e-14, ...,
        7.08615429e+01,  2.97284454e+01,  2.87756373e+02])
  status: 0
 success: True
       x: array([9.22000000e-02, 2.84400000e-01, 1.17510000e+00, ...,
       6.22343530e-04, 8.40889971e-04, 2.41336186e-01])
529239.4682509315


### Finding  the top 10 most affected itinerary in terms of the optimal seat allocation.
1. Difference between the parameters in both the cases are calculated and arranged it in acendind order 
2. So the last 10 itinerary are the most effected ones 

In [20]:
res_diff = res.x - res_new.x

In [21]:
res_diff_abs = np.absolute(res_diff)

In [22]:
fare_dup['res_diff_abs'] = res_diff_abs

In [23]:
fare_dup.sort_values(["res_diff_abs"], axis=0, 
                 ascending=True, inplace=True) 

In [24]:
fare_dup

,itinerary_name,fare,res_diff_abs
7356,21059142**I******20020920,940.4400,0.000000
7321,21058599CHIDXBGVA20020920,1204.8000,0.000000
7322,21058619AEIDXBGVA20020920,1889.7200,0.000000
7323,21058619CHIDXBGVA20020920,2502.8700,0.000000
7324,21058638CHIDXBGVA20020920,351.0500,0.000000
7325,21058643**I******20020920,2293.2375,0.000000
7326,21058643CHIDXBGVA20020920,2123.3875,0.000000
7327,21058656**I******20020920,335.3800,0.000000
7328,21058656AEIDXBGVA20020920,382.9200,0.000000
1721,43550**G******20020920,0.0000,0.000000
